<a href="https://colab.research.google.com/github/GHC-av/DL_project/blob/main/CNN_WeightsTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning - Final Project
## CNN Weights Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import keras
from keras.models import Sequential
from keras import layers
from keras import models
import sklearn
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
#from keras.preprocessing.image import array_to_img
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image_dataset_from_directory

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
os.listdir("/content/drive/My Drive/CoronaHack")

['Chest_xray_Corona_Metadata.csv',
 'Chest_xray_Corona_dataset_Summary.csv',
 'archive.zip',
 'Coronahack-Chest-XRay-Dataset',
 'weight_14nov.h5',
 'modelDN_15nov.h5',
 'modelCNN_15nov.h5',
 'modelCNN_15novV2.h5',
 'modelDN_15novV2.h5',
 'weightDN_15nov.h5']

# Data set for transfer learning 

https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset



In [ ]:
#!unzip -uq "/content/drive/My Drive/CoronaHack/archive.zip" -d "/content/drive/My Drive/CoronaHack"

In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/CoronaHack/Chest_xray_Corona_Metadata.csv")
train_df.shape

(5910, 6)

In [ ]:
train_df.dropna(how = 'all')
train_df.isnull().sum()

Unnamed: 0                   0
X_ray_image_name             0
Label                        0
Dataset_type                 0
Label_2_Virus_category    5841
Label_1_Virus_category    1576
dtype: int64

In [ ]:
train_df.fillna('unknown', inplace=True)
train_df.isnull().sum()

Unnamed: 0                0
X_ray_image_name          0
Label                     0
Dataset_type              0
Label_2_Virus_category    0
Label_1_Virus_category    0
dtype: int64

In [ ]:
train = train_df[train_df['Dataset_type'] == 'TRAIN']
test = train_df[train_df['Dataset_type'] == 'TEST']

#assert train_data.shape[0] + test_data.shape[0] == train_df.shape[0]

print(f"Shape of train data : {train.shape}")
print(f"Shape of test data : {test.shape}")


Shape of train data : (5286, 6)
Shape of test data : (624, 6)


In [ ]:
train.sample(10)

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
1308,1308,NORMAL2-IM-1401-0001.jpeg,Normal,TRAIN,unknown,unknown
593,593,NORMAL2-IM-0394-0001.jpeg,Normal,TRAIN,unknown,unknown
2938,2938,person25_bacteria_118.jpeg,Pnemonia,TRAIN,unknown,bacteria
829,829,NORMAL2-IM-0804-0001.jpeg,Normal,TRAIN,unknown,unknown
1303,1303,NORMAL2-IM-1423-0001.jpeg,Normal,TRAIN,unknown,unknown
3522,3522,person354_bacteria_1635.jpeg,Pnemonia,TRAIN,unknown,bacteria
4023,4023,person491_bacteria_2073.jpeg,Pnemonia,TRAIN,unknown,bacteria
3875,3875,person397_virus_790.jpeg,Pnemonia,TRAIN,unknown,Virus
2252,2252,person1367_virus_2351.jpeg,Pnemonia,TRAIN,unknown,Virus
1787,1787,person1186_virus_2022.jpeg,Pnemonia,TRAIN,unknown,Virus


In [ ]:
test.sample(10)

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5708,5731,person100_bacteria_481.jpeg,Pnemonia,TEST,unknown,bacteria
5813,5836,person44_virus_93.jpeg,Pnemonia,TEST,unknown,Virus
5573,5596,person147_bacteria_705.jpeg,Pnemonia,TEST,unknown,bacteria
5321,5344,NORMAL2-IM-0012-0001.jpeg,Normal,TEST,unknown,unknown
5783,5806,person74_virus_135.jpeg,Pnemonia,TEST,unknown,Virus
5761,5784,person81_bacteria_398.jpeg,Pnemonia,TEST,unknown,bacteria
5887,5910,person1660_virus_2869.jpeg,Pnemonia,TEST,unknown,Virus
5736,5759,person8_virus_27.jpeg,Pnemonia,TEST,unknown,Virus
5629,5652,person128_bacteria_605.jpeg,Pnemonia,TEST,unknown,bacteria
5754,5777,person83_bacteria_410.jpeg,Pnemonia,TEST,unknown,bacteria


In [ ]:
train_pn = train.loc[train.Label == 'Pnemonia', 'Label'].count()
train_n = train.loc[train.Label == 'Normal', 'Label'].count()

print(f"there are {train_pn} pneumonia cases in the train data set")
print(f"there are {train_n} normal cases in the train data set")

test_pn = test.loc[test.Label == 'Pnemonia', 'Label'].count()
test_n = test.loc[test.Label == 'Normal', 'Label'].count()

print(f"there are {test_pn} pneumonia cases in the test data set")
print(f"there are {test_n} normal cases in the test data set")



there are 3944 pneumonia cases in the train data set
there are 1342 normal cases in the train data set
there are 390 pneumonia cases in the test data set
there are 234 normal cases in the test data set


In [ ]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
    ]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )

val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_path = "/content/drive/My Drive/CoronaHack/Coronahack-Chest-XRay-Dataset/train"
test_path = "/content/drive/My Drive/CoronaHack/Coronahack-Chest-XRay-Dataset/test"

In [ ]:
img_size = 150
batch_size = 32

train_set = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_path,
    x_col="X_ray_image_name",
    y_col="Label",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size= batch_size,
    seed=25,
    shuffle=True
)

val_set = val_datagen.flow_from_dataframe(
    dataframe=test,
    directory=test_path,
    x_col="X_ray_image_name",
    y_col="Label",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size= batch_size,
    seed=25,
    shuffle=True
)

Found 5286 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [ ]:
pre_model = models.Sequential([
                    layers.Conv2D(filters=32, kernel_size=3, strides=1, activation="relu",
                           input_shape=[img_size,img_size,3]),
                    layers.MaxPool2D(pool_size=2, strides=2),
                    layers.Conv2D(filters=64, kernel_size=3, strides=1, activation="relu"),
                    layers.MaxPool2D(pool_size=2, strides=2),
                    layers.Conv2D(filters=128, kernel_size=3, strides=1, activation="relu"),
                    layers.MaxPool2D(pool_size=2, strides=2),
                    layers.Conv2D(filters=128, kernel_size=3, strides=1, activation="relu"),
                    layers.MaxPool2D(pool_size=2, strides=2),
                    layers.Flatten(),
                    layers.Dropout(rate=0.5),
                    layers.Dense(units=512, activation="relu"),
                    layers.Dense(units=1, activation="sigmoid")
])

In [ ]:
pre_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=METRICS
    )

In [ ]:
weight_pn = (1 / train_pn)*(train_pn + train_n)/2.0
weight_n = (1 / train_n)*(train_pn + train_n)/2.0

# Order of the classes, mapped to the label indices, is alphanumeric
class_weight = {0: weight_n, 1: weight_pn}

print('Weight for class pneumonia: {:.2f}'.format(weight_pn)) 
print('Weight for class normal: {:.2f}'.format(weight_n))

Weight for class pneumonia: 0.67
Weight for class normal: 1.97


In [ ]:
train_count = train.shape[0]
test_count = test.shape[0]

In [ ]:
history = pre_model.fit(train_set,
                    steps_per_epoch = train_count // batch_size,
                    epochs = 50,
                    validation_data = val_set,
                    class_weight = class_weight,
                    validation_steps = test_count // batch_size
                    )

pre_model.save_weights('/content/drive/My Drive/CoronaHack/weightCNN.h5')

In [ ]:
#Iterate thru all the layers of the model
for layer in pre_model.layers:
    if 'conv' in layer.name:
        weights, bias= layer.get_weights()
        
        #normalize filter values between  0 and 1 for visualization
        f_min, f_max = weights.min(), weights.max()
        filters = (weights - f_min) / (f_max - f_min)
        print(layer.name, filters.shape)
        print(filters.shape[3])
        filter_cnt=1
        
        #plotting all the filters
        for i in range(filters.shape[3]):
            #get the filters
            filt=filters[:,:,:, i]
            #plotting each of the channel, color image RGB channels
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:,:, j])
                filter_cnt+=1
        plt.show()